In [1]:
from google.colab import files

# Upload your missing PDF files manually
uploaded = files.upload()

Saving 04232021-ommu-update.pdf to 04232021-ommu-update.pdf
Saving 05102019-ommu-update.pdf to 05102019-ommu-update.pdf
Saving 05132022-ommu-update.pdf to 05132022-ommu-update.pdf
Saving 05172019-ommu-update.pdf to 05172019-ommu-update.pdf
Saving 05202022-ommu-update.pdf to 05202022-ommu-update.pdf
Saving 05212021-ommu-update.pdf to 05212021-ommu-update.pdf
Saving 05242019-ommu-update.pdf to 05242019-ommu-update.pdf
Saving 05282021-ommu-update.pdf to 05282021-ommu-update.pdf
Saving 05312019-ommu-update.pdf to 05312019-ommu-update.pdf
Saving 06072019-ommu-update.pdf to 06072019-ommu-update.pdf
Saving 06142019-ommu-update.pdf to 06142019-ommu-update.pdf
Saving 06212019-ommu-update.pdf to 06212019-ommu-update.pdf
Saving 06282021-ommu-update.pdf to 06282021-ommu-update.pdf
Saving 07052019-ommu-update.pdf to 07052019-ommu-update.pdf
Saving 07122019-ommu-update.pdf to 07122019-ommu-update.pdf
Saving 07162019-ommu-update.pdf to 07162019-ommu-update.pdf
Saving 07192019-ommu-update.pdf to 07192

In [9]:
import os
import PyPDF2
import pandas as pd
from datetime import datetime

def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def extract_table_from_text(text, filename):
    lines = text.split('\n')
    table_data = []
    headers = ["MMTC Name", "Dispensing Locations", "Medical Marijuana (mgs THC)",
               "Low-THC Cannabis (mgs CBD)", "Marijuana in a Form for Smoking (oz)"]
    table_started = False
    current_mmtc = ""

    print(f"Attempting to extract data from {filename}")

    for i, line in enumerate(lines):
        if "MMTC Dispensations for" in line or "MMTC Name" in line:
            table_started = True
            print(f"Table start detected at line {i}: {line}")
            continue

        if table_started:
            parts = line.split()
            if len(parts) >= 5:
                try:
                    smoking = float(parts[-1].replace(',', '')) if parts[-1] != 'N/A' else 0
                    low_thc = int(parts[-2].replace(',', '')) if parts[-2] != 'N/A' else 0
                    medical_marijuana = int(parts[-3].replace(',', '')) if parts[-3] != 'N/A' else 0
                    locations = int(parts[-4]) if parts[-4] != 'N/A' and parts[-4] != 'Delivery' else 0
                    mmtc_name = " ".join(parts[:-4])

                    if current_mmtc:
                        mmtc_name = f"{current_mmtc} {mmtc_name}"
                        current_mmtc = ""

                    # Adjust MMTC names
                    mmtc_name = mmtc_name.replace("Welln ess", "Wellness")
                    mmtc_name = mmtc_name.replace("Cannabis Dispensary", "Ayr")
                    mmtc_name = mmtc_name.replace("the Planet", "13th Planet")
                    mmtc_name = mmtc_name.replace("- Cannabis for Real Life", "Insa")

                    table_data.append([mmtc_name, locations, medical_marijuana, low_thc, smoking])
                    print(f"Processed row: {mmtc_name}")
                except ValueError as e:
                    if "Total" in line:
                        print(f"Table end detected at line {i}: {line}")
                        break
                    current_mmtc = " ".join(parts)
                    print(f"Partial row detected: {current_mmtc}")
            elif "Total" in line:
                print(f"Table end detected at line {i}: {line}")
                break

    print(f"Extracted {len(table_data)} rows of data from {filename}")
    return headers, table_data

def process_data(data, headers, report_date):
    if not data:
        print("No data to process")
        return None
    df = pd.DataFrame(data, columns=headers)
    df['Report Date'] = report_date
    return df

def extract_date_from_filename(filename):
    # Assuming filename format is MMDDYYYY-ommu-update.pdf
    date_str = filename.split('-')[0]
    return datetime.strptime(date_str, "%m%d%Y").strftime("%Y-%m-%d")

def process_single_report(file_path):
    report_date = extract_date_from_filename(os.path.basename(file_path))

    with open(file_path, 'rb') as file:
        pdf_text = extract_text_from_pdf(file)
        headers, table_data = extract_table_from_text(pdf_text, os.path.basename(file_path))
        if table_data:
            return process_data(table_data, headers, report_date)
        else:
            print(f"No data extracted from {file_path}")
            return None

# Main execution
if __name__ == "__main__":
    combined_data = pd.DataFrame()  # Create an empty DataFrame to hold all data

    # Get all PDF files in the current directory
    pdf_files = [f for f in os.listdir() if f.endswith('.pdf')]

    for pdf_file in pdf_files:
        print(f"\nProcessing report {pdf_file}...")
        processed_data = process_single_report(pdf_file)

        if processed_data is not None:
            combined_data = pd.concat([combined_data, processed_data], ignore_index=True)

    # After processing all files, save the combined data to CSV
    if not combined_data.empty:
        combined_data.to_csv('combined_cannabis_data.csv', index=False)
        print("\nAll reports processed and saved to 'combined_cannabis_data.csv'")
        print(f"Total rows in combined data: {len(combined_data)}")
        print("\nSample of combined data:")
        print(combined_data.head())
    else:
        print("No data was processed.")


Processing report 05212021-ommu-update.pdf...
Attempting to extract data from 05212021-ommu-update.pdf
Table start detected at line 67: MMTC Dispensations for May 14 – May 20, 2021 : 
Table start detected at line 68: MMTC Name  Dispensing 
Partial row detected: (mgs CBD) Marijuana in a Form
Processed row: (mgs CBD) Marijuana in a Form Trulieve
Processed row: Surterra Wellness
Processed row: Curaleaf
Processed row: Liberty Health Sciences
Processed row: AltMed Florida (MüV)
Processed row: Fluent
Processed row: VidaCann
Processed row: GrowHealthy
Processed row: Columbia Care Florida
Processed row: MedMen
Processed row: One Plant
Processed row: Harvest
Processed row: GTI (Rise Dispensaries)
Processed row: The Botanist
Processed row: Sanctuary Medicinals, LLC
Processed row: Insa
Processed row: Revolution Florida
Processed row: Perkins Nursery, Inc
Processed row: Green Ops Group FL, LLC
Processed row: The Flowery
Processed row: Green Dragon
Processed row: Cookies Florida, Inc.
Processed ro